<a href="https://colab.research.google.com/github/dylandeppiesse/realty-project/blob/main/Realty_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
from requests import get
import time as t
from random import randint
import pandas as pd
import itertools
import re

In [ ]:
# setting headers which will make it look like this is coming
# from an actual browser instead of a scraper
headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML,\
            like Gecko) Chrome/41.0.2228.0 Safari/537.36'})

In [ ]:
realtor = 'https://www.rew.ca/sitemap/real-estate/bc'
response = get(realtor, headers=headers)
html_soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
%%time
#setting up a list to get every sub-region
subregion = []
bc_urls = 'https://www.rew.ca/sitemap/real-estate/bc'
bc_r = get(bc_urls,headers = headers)
bc_html = BeautifulSoup(bc_r.text, 'html.parser')
subregion_containers = bc_html.find_all('div', class_ ='gridblock-item')

CPU times: user 80 ms, sys: 1.04 ms, total: 81.1 ms
Wall time: 701 ms


In [ ]:
len(subregion_containers)

177

In [ ]:
if subregion_containers != []:
    for container in subregion_containers:
        url = container.find('a', class_= 'gridblock-link')
        subregion.append(url)

t.sleep(randint(1,3))

print('Scraped {} links.'.format(len(subregion)))

Scraped 177 links.


In [ ]:
#converting subregions to strings to allow regex 
subregion_str=[str(region) for region in subregion]

#performing regex to remove unecessary info
subregion_urls = []
for item in subregion_str:
    subregion_urls.append(re.findall("href=(.+?)>",item))

In [ ]:
#convert to list of STR to perform changes on
subregion_url_str=[str(region) for region in subregion_urls]

#cleaning up subregion_urls so that we can insert them in a loop to get Neigbourhoods
#this loop needs to look like rew.ca + subregion_url
clean_urls = []
for region in subregion_url_str:
    clean_urls.append(region.replace('"','').replace("['",'').replace("']",''))

In [ ]:
%%time 
#loop to get neighbourhoods from subregions list above
cities = []
for url in clean_urls:
    subregion_url = 'https://www.rew.ca'+ url
    r = get(subregion_url, headers=headers)
    city_html = BeautifulSoup(r.text, 'html.parser')
    city_container = city_html.find_all('div', class_ = 'gridblock-item')
    
    if city_container != []:
        for container in city_container:
            city = container.find_all('a', class_ = 'gridblock-link')
            cities.append(city)

    t.sleep(randint(1,3))

CPU times: user 8.88 s, sys: 506 ms, total: 9.39 s
Wall time: 8min 12s


In [ ]:
print(len(cities))
print(cities[0])

685
[<a class="gridblock-link" href="/sitemap/real-estate/belgo-black-mountain-kelowna-bc">Belgo - Black Mountain
(26)
</a>]


In [ ]:
#converting cities to strings to allow regex 
city_str=[str(city) for city in cities]

#performing regex to remove unecessary info
city_urls = []
for item in city_str:
    city_urls.append(re.findall("href=(.+?)>",item))

In [ ]:
#convert to list of STR to perform changes on
city_url_str=[str(city) for city in city_urls]

#cleaning up city_urls so that we can insert them in a loop to get Neigbourhoods
#this loop needs to look like rew.ca + subregion_url
clean_city_urls = []
for city in city_url_str:
    clean_city_urls.append(city.replace('"','').replace("['",'').replace("']",''))

In [ ]:
city_list = []
for city in clean_city_urls:
    city_list.append(city[20:])

In [ ]:
city_list[:3]

['/belgo-black-mountain-kelowna-bc',
 '/central-city-kelowna-bc',
 '/glenmore-clifton-dilworth-kelowna-bc']

In [ ]:
# setting up lists to form our dataframe later on
address = []
city = []
price = []
bed_bath_sqft = []
property_type = []

In [ ]:
# looks like all the cities work and are in our clean_city_urls list, we now need to get the actual listings (I think)
# we need the URL to look like 'https://www.rew.ca/developments/areas' + city_list
%%time 

#loop to get individual listings from city_list above
listings = []
for url in city_list:
    n_pages = 0
    for page in range(1,30):
        listing_url = 'https://www.rew.ca/developments/areas'+ url + "/page/" + str(page)
        r = get(listing_url, headers=headers)
        listing_html = BeautifulSoup(r.text, 'html.parser')
        listing_container = listing_html.find_all('div', class_ = 'diplaypanel-content')
    
        if listing_container != []:
            for container in listing_container:

                #address
                location = container.find_all('div', class_ = 'displaypanel-section')[0].text
                address.append(location)

                #city
                zone = container.find_all('ul', class_ = 'l-pipedlist displaypanel-info')
                city.append(zone)

                #price
                amt = int(container.find('div', class_ = 'displaypanel-title').text[2:].replace(',',''))
                price.append(amt)

                #Beds, Baths, and Square Footage
                living_space = container.find_all('ul', class_ = 'l-pipedlist')[1].text
                bed_bath_sqft.append(living_space)

                #property type
                p_type = container.find_all('div', class_ = 'displaypanel-info')[0].text
                property_type.append(p_type)

        else:
           break

        t.sleep(randint(1,3))

print('Scraped {} listings'.format(len(listings)))

Scraped 0 listings
CPU times: user 10.5 s, sys: 424 ms, total: 11 s
Wall time: 45.8 s


In [ ]:
#above code isn't working, break it down into steps and figure out if the loops are working